In [ ]:

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import json
from datetime import datetime, timezone

def weatherapi(latitude, longitude,location_name):
    # Setup session with caching and retry logic
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
    client = openmeteo_requests.Client(session=retry_session)

    # API endpoint and parameters
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature","wind_speed_10m",
            "precipitation_probability", "precipitation", "rain", "showers", "snowfall",
            "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "visibility",
            "evapotranspiration", "et0_fao_evapotranspiration", "vapour_pressure_deficit"
        ]
    }

    # Fetch weather data
    responses = client.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    # Generate datetime range for all hourly data points
    time_range = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Find current UTC hour rounded down
    now_utc = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)

    # Find the index of the current hour in the time_range
    try:
        current_index = time_range.get_loc(now_utc)
    except KeyError:
        # If current time not in forecast range, fallback to nearest time (first available)
        current_index = 0

    # Prepare data for the current hour only
    current_hour_data = {"city": location_name}

    variables = [
        "temperature_celcius", "humidity_%", "dew_temperature_celcius", "feels_like_temperature_celcius","wind_speed_kmph",
        "precipitation_%", "precipitation_occured_mm", "rain_mm", "showers_mm", "snowfall_mm",
        "snow_depth_mm", "weather_code", "mean_sea_level_pressure_hpa", "surface_pressure_hpa", "cloud_cover_%",
        "visibility_m",
        "evapotranspiration_mm", "et0_fao_evapotranspiration_mm", "vapour_pressure_deficit_kpa"
    ]

    # Extract the value for the current hour from each variable
    for i, var in enumerate(variables):
        values = hourly.Variables(i).ValuesAsNumpy()
        current_hour_data[var] = float(values[current_index]) if values.size > current_index else None

    return current_hour_data

def fetch_weather_batch():
    delhi=weatherapi(28.7041,77.1025,'Delhi')

    mumbai=weatherapi(18.9582,72.8321,'Mumbai')

    bengaluru=weatherapi(12.9629,77.5775,'Bengaluru')

    hyderabad=weatherapi(17.4065, 78.4772,'Hyderabad')

    chennai=weatherapi(13.0843,80.2705,'Chennai')

    kolkata=weatherapi(22.5744,88.3629,'Kolkata')

    ahmedabad=weatherapi(23.0225,72.5714,'Ahmedabad')

    pune=weatherapi(18.5246,73.8786,'Pune')

    jaipur=weatherapi(26.9124,75.7873,'Jaipur')

    lucknow=weatherapi(26.8467,80.9462,'Lucknow')
    
    all_cities=[delhi,mumbai,bengaluru,hyderabad,chennai,kolkata,ahmedabad,pune,jaipur,lucknow]

    # debugging
    print(all_cities)


    return all_cities

fetch_weather_batch()











"""
    Fetches current hourly weather data for a specified location using the Open-Meteo API.
    
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        location_name (str): Human-readable name of the location (for reference).
        
    Returns:
        dict: Weather data for the current hour, including temperature, humidity, precipitation,
              cloud cover, and other atmospheric variables.
              
    Implementation details:
        - Uses a cached session with automatic retries to improve API request reliability.
        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.
        - Converts API response timestamps to pandas datetime for easier time handling.
        - Extracts the data corresponding to the current UTC hour.
        - Returns a dictionary with all requested weather parameters for the current hour.

        
Data Dictionary Explanation:

- location: Name of the city (string).
- date: Date and time of the weather reading in 'YYYY-MM-DD HH:MM:SS' format (string).
- temperature_2m: Air temperature measured 2 meters above the ground, in degrees Celsius (float).
- relative_humidity_2m: Relative humidity at 2 meters height, in percentage (%) (float).
- dew_point_2m: Dew point temperature at 2 meters height, in degrees Celsius (float).
- apparent_temperature: Feels-like temperature considering humidity and wind, in degrees Celsius (float).
- wind_speed_10m: wind speed in kmph above 10 meter of ground level(float).
- precipitation_probability: Probability of precipitation occurring, in percentage (%) (float).
- precipitation: Amount of precipitation expected or recorded, in millimeters (mm) (float).
- rain: Amount of rain specifically, in millimeters (mm) (float).
- showers: Amount of showers, in millimeters (mm) (float).
- snowfall: Amount of snowfall, in millimeters (mm) (float).
- snow_depth: Depth of snow on the ground, in millimeters (mm) (float).
- weather_code: Numerical code representing weather conditions (e.g., clear, rain, snow) (float).
- pressure_msl: Atmospheric pressure at mean sea level, in hectopascals (hPa) (float).
- surface_pressure: Atmospheric pressure at the surface, in hectopascals (hPa) (float).
- cloud_cover: Total cloud cover percentage (%) (float).
- cloud_cover_low: Low-level cloud cover percentage (%) (float).
- cloud_cover_mid: Mid-level cloud cover percentage (%) (float).
- cloud_cover_high: High-level cloud cover percentage (%) (float).
- visibility: Visibility distance in meters (float).
- evapotranspiration: Amount of evapotranspiration, representing water transfer from land to atmosphere, in millimeters (mm) (float).
- et0_fao_evapotranspiration: Reference evapotranspiration (FAO standard), in millimeters (mm) (float).
- vapour_pressure_deficit: Difference between saturation and actual vapor pressure, indicating dryness of air, in kilopascals (kPa) (float).

"""


[{'city': 'Delhi', 'temperature_celcius': 28.678499221801758, 'humidity_%': 90.0, 'dew_temperature_celcius': 26.874582290649414, 'feels_like_temperature_celcius': 35.580322265625, 'wind_speed_kmph': 7.695919990539551, 'precipitation_%': 28.0, 'precipitation_occured_mm': 0.0, 'rain_mm': 0.0, 'showers_mm': 0.0, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 3.0, 'mean_sea_level_pressure_hpa': 999.5999755859375, 'surface_pressure_hpa': 974.8541259765625, 'cloud_cover_%': 100.0, 'visibility_m': 39.0, 'evapotranspiration_mm': 56.0, 'et0_fao_evapotranspiration_mm': 100.0, 'vapour_pressure_deficit_kpa': 24140.0}, {'city': 'Mumbai', 'temperature_celcius': 28.415000915527344, 'humidity_%': 88.0, 'dew_temperature_celcius': 26.233631134033203, 'feels_like_temperature_celcius': 33.040435791015625, 'wind_speed_kmph': 20.188907623291016, 'precipitation_%': 95.0, 'precipitation_occured_mm': 0.800000011920929, 'rain_mm': 0.0, 'showers_mm': 0.800000011920929, 'snowfall_mm': 0.0, 'snow_depth_

"\n    Fetches current hourly weather data for a specified location using the Open-Meteo API.\n    \n    Args:\n        latitude (float): Latitude of the location.\n        longitude (float): Longitude of the location.\n        location_name (str): Human-readable name of the location (for reference).\n        \n    Returns:\n        dict: Weather data for the current hour, including temperature, humidity, precipitation,\n              cloud cover, and other atmospheric variables.\n              \n    Implementation details:\n        - Uses a cached session with automatic retries to improve API request reliability.\n        - Fetches hourly forecast data from Open-Meteo API for a wide range of weather variables.\n        - Converts API response timestamps to pandas datetime for easier time handling.\n        - Extracts the data corresponding to the current UTC hour.\n        - Returns a dictionary with all requested weather parameters for the current hour.\n\n        \nData Dictionary E

In [124]:
import pandas as pd
import random
from datetime import datetime
# from weather_monitor import weatherapi,fetch_weather_batch


df = pd.DataFrame(fetch_weather_batch())


#rounding off all float cols
df = df.round({
    'temperature_celcius': 2,
    'humidity_%': 0,
    'dew_temperature_celcius': 2,
    'feels_like_temperature_celcius': 2,
    'wind_speed_kmph':2,
    'precipitation_%': 0,
    'precipitation_occured_mm': 2,
    'rain_mm': 2,
    'showers_mm': 2,
    'snowfall_mm': 2,
    'snow_depth_mm': 2,
    'mean_sea_level_pressure_hpa': 2,
    'surface_pressure_hpa': 1,
    'cloud_cover_%': 0,
    'visibility_m': 2,
    'evapotranspiration_mm': 3,
    'et0_fao_evapotranspiration_mm': 3,
    'vapour_pressure_deficit_kpa': 4
})


#add water strees index
df['water_stress_index'] = (
    df['evapotranspiration_mm'] / (df['precipitation_occured_mm'] + 0.01)  # Avoid div/0
).round(2)


# Add current datetime in the specified format
df['created_at'] = datetime.now()


#add column of check extreme condition or not
# List of dangerous/extreme weather codes
extreme_codes = []
if extreme_codes:
    df['extreme_weather_yn'] = df['weather_code'].isin(extreme_codes).map({True: 'Y', False: 'N'})
else:
    df['extreme_weather_yn'] = 'N/A'



#snowfall_yn column
df['snowfall_yn'] = 'N'
df.loc[df['snowfall_mm'] > 0, 'snowfall_yn'] = 'Y'


# rainfall_yn column
df['rainfall_yn'] = 'N'
df.loc[(df['rain_mm'] > 0) | (df['showers_mm'] > 0), 'rainfall_yn'] = 'Y'


# foggy_yn column
df['foggy_yn'] = 'N'
df.loc[df['visibility_m'] < 1000, 'foggy_yn'] = 'Y'


# effective_precipitation_mm based on sum of snowfall,rain and shower i mm
SNOW_TO_LIQUID_RATIO = 10
# Convert snowfall_mm to its water equivalent
df['snowfall_water_equivalent_mm'] = df['snowfall_mm'] / SNOW_TO_LIQUID_RATIO
df['effective_precipitation_mm'] = df['rain_mm'] + df['showers_mm'] + df['snowfall_water_equivalent_mm']


#creating unique_id
city_short_map = {
    'Delhi': 'DEL', 
    'Mumbai': 'MUM', 
    'Bengaluru': 'BLR', 
    'Hyderabad': 'HYD', 
    'Chennai': 'CHE',  
    'Kolkata': 'KOL',  
    'Ahmedabad': 'AMD',  
    'Pune': 'PUN',  
    'Jaipur': 'JAI', 
    'Lucknow': 'LKO'  
}
def generate_unique_id(row):
    city_short = city_short_map.get(row['city'], row['city'][:3].upper())
    weather_code_str = f"{int(row['weather_code']):02d}"  # pad weather code to 2 digits
    rand_num = f"{random.randint(0, 99):02d}"            # 2 digit random number
    dt = row['created_at']
    dt_str = dt.strftime('%d%m%y%H%M%S')                # DDMMYYHHMMSS 
    ms_str = f"{int(dt.microsecond / 10000):02d}"        # first 2 digits of milliseconds  
    unique_id = f"{city_short}-{weather_code_str}-{rand_num}-{dt_str}{ms_str}"
    return unique_id
df['unique_id'] = df.apply(generate_unique_id, axis=1)
#unique id = 3 leters city shortform-weather_code-random 2 digit-date time in format DDMMYYHHMMSSMS


# #convert to boolean field if needed
# for col in ['rainfall_yn', 'snowfall_yn', 'foggy_yn']:
#     df[col] = df[col].map({'Y': True, 'N': False})


#weather type column based on weather code (WMO standard)
# Step 1: Create the mapping dictionary
weather_code_map = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    4: "Fog",
    5: "Drizzle",
    6: "Rain",
    7: "Showers",
    8: "Snow",
    9: "Rain and snow",
    10: "Sleet",
    11: "Hail",
    12: "Thunderstorm",
    13: "Duststorm",
    14: "Sandstorm",
    15: "Smoke",
    16: "Volcanic ash",
    17: "Windstorm",
    18: "Tornado",
    19: "Freezing rain",
    20: "Mist",
    21: "Light rain",
    22: "Moderate rain",
    23: "Heavy rain",
    24: "Light snow",
    25: "Moderate snow",
    26: "Heavy snow",
    27: "Light sleet",
    28: "Moderate sleet",
    29: "Heavy sleet",
    30: "Light hail",
    31: "Moderate hail",
    32: "Heavy hail",
    33: "Light thunderstorm",
    34: "Moderate thunderstorm",
    35: "Heavy thunderstorm",
    36: "Light duststorm",
    37: "Moderate duststorm",
    38: "Heavy duststorm",
    39: "Light sandstorm",
    40: "Moderate sandstorm",
    41: "Heavy sandstorm",
    42: "Light smoke",
    43: "Moderate smoke",
    44: "Heavy smoke",
    45: "Light volcanic ash",
    46: "Moderate volcanic ash",
    47: "Heavy volcanic ash",
    48: "Light windstorm",
    49: "Strong windstorm",
    50: "Severe windstorm",
    51: "Light tornado",
    52: "Moderate tornado",
    53: "Heavy tornado",
    54: "Light freezing rain",
    55: "Moderate freezing rain",
    56: "Heavy freezing rain",
    57: "Light mist",
    58: "Moderate mist",
    59: "Heavy mist",
    60: "Light rain and snow",
    61: "Moderate rain and snow",
    62: "Heavy rain and snow",
    63: "Light sleet and snow",
    64: "Moderate sleet and snow",
    65: "Heavy sleet and snow",
    66: "Light hail and snow",
    67: "Moderate hail and snow",
    68: "Heavy hail and snow",
    69: "Light thunderstorm with rain",
    70: "Moderate thunderstorm with rain",
    71: "Heavy thunderstorm with rain",
    72: "Light thunderstorm with snow",
    73: "Moderate thunderstorm with snow",
    74: "Heavy thunderstorm with snow",
    75: "Light thunderstorm with sleet",
    76: "Moderate thunderstorm with sleet",
    77: "Heavy thunderstorm with sleet",
    78: "Light thunderstorm with hail",
    79: "Moderate thunderstorm with hail",
    80: "Heavy thunderstorm with hail",
    81: "Light thunderstorm with dust",
    82: "Moderate thunderstorm with dust",
    83: "Heavy thunderstorm with dust",
    84: "Light thunderstorm with sand",
    85: "Moderate thunderstorm with sand",
    86: "Heavy thunderstorm with sand",
    87: "Light thunderstorm with smoke",
    88: "Moderate thunderstorm with smoke",
    89: "Heavy thunderstorm with smoke",
    90: "Light thunderstorm with volcanic ash",
    91: "Moderate thunderstorm with volcanic ash",
    92: "Heavy thunderstorm with volcanic ash",
    93: "Light windstorm with rain",
    94: "Strong windstorm with rain",
    95: "Severe windstorm with rain",
    96: "Light windstorm with snow",
    97: "Strong windstorm with snow",
    98: "Severe windstorm with snow",
    99: "Light windstorm with sleet",
    100: "Severe windstorm with sleet"
}
df['weather_type'] = df['weather_code'].map(weather_code_map)


# drop if null available in important columns-
df = df.dropna(subset=['city','temperature_celcius', 'humidity_%', 'wind_speed_kmph','weather_code','precipitation_occured_mm','created_at','extreme_weather_yn'])


#ordering columns
sorted_columns = [
    # Location Info
    'city',   
    # Core Weather Measurements
    'temperature_celcius',
    'feels_like_temperature_celcius',
    'dew_temperature_celcius',
    'humidity_%',
    'vapour_pressure_deficit_kpa',
    'wind_speed_kmph',    
    # Precipitation & Snowfall
    'precipitation_%',
    'precipitation_occured_mm',
    'rain_mm',
    'showers_mm',
    'rainfall_yn',
    'snowfall_mm',
    'snow_depth_mm',
    'snowfall_yn',
    'effective_precipitation_mm',
    # Fog & Visibility
    'foggy_yn',
    'visibility_m',
    'cloud_cover_%',
    # Pressure & Wind
    'mean_sea_level_pressure_hpa',
    'surface_pressure_hpa',
    # Evaporation
    'evapotranspiration_mm',
    'et0_fao_evapotranspiration_mm',
    # Weather Code & Classification
    'weather_code',
    'weather_type',   
    # Environmental Indicators
    'water_stress_index',
    'extreme_weather_yn',
    # 🕒 Timestamps
    'unique_id',
    'created_at'
]
df = df[sorted_columns]

# for debugging
# print(df)

[{'city': 'Delhi', 'temperature_celcius': 28.12849998474121, 'humidity_%': 90.0, 'dew_temperature_celcius': 26.33185577392578, 'feels_like_temperature_celcius': 34.616050720214844, 'wind_speed_kmph': 7.895416259765625, 'precipitation_%': 40.0, 'precipitation_occured_mm': 2.299999952316284, 'rain_mm': 0.0, 'showers_mm': 2.299999952316284, 'snowfall_mm': 0.0, 'snow_depth_mm': 0.0, 'weather_code': 80.0, 'mean_sea_level_pressure_hpa': 999.5, 'surface_pressure_hpa': 974.712158203125, 'cloud_cover_%': 100.0, 'visibility_m': 61.0, 'evapotranspiration_mm': 53.0, 'et0_fao_evapotranspiration_mm': 100.0, 'vapour_pressure_deficit_kpa': 24140.0}, {'city': 'Mumbai', 'temperature_celcius': 28.26500129699707, 'humidity_%': 88.0, 'dew_temperature_celcius': 26.086030960083008, 'feels_like_temperature_celcius': 33.01593017578125, 'wind_speed_kmph': 18.626304626464844, 'precipitation_%': 98.0, 'precipitation_occured_mm': 0.800000011920929, 'rain_mm': 0.0, 'showers_mm': 0.800000011920929, 'snowfall_mm': 0.

In [127]:
import pandas as pd

# Show more columns
pd.set_option('display.max_columns', None)

# Optionally, show more rows (if you also need that)
pd.set_option('display.max_rows', None)

# Adjust column width (optional)
pd.set_option('display.max_colwidth', None)

# Prevent horizontal truncation
pd.set_option('display.width', None)  # Auto-detect terminal width

df
# df.isnull().sum()
# df.dtypes
# print("Total columns:", len(df.columns))


,city,temperature_celcius,feels_like_temperature_celcius,dew_temperature_celcius,humidity_%,vapour_pressure_deficit_kpa,wind_speed_kmph,precipitation_%,precipitation_occured_mm,rain_mm,showers_mm,rainfall_yn,snowfall_mm,snow_depth_mm,snowfall_yn,effective_precipitation_mm,foggy_yn,visibility_m,cloud_cover_%,mean_sea_level_pressure_hpa,surface_pressure_hpa,evapotranspiration_mm,et0_fao_evapotranspiration_mm,weather_code,weather_type,water_stress_index,extreme_weather_yn,unique_id,created_at
0,Delhi,28.13,34.62,26.33,90.0,24140.0,7.90,40.0,2.3,0.0,2.3,Y,0.0,0.0,N,2.3,Y,61.0,100.0,999.5,974.7,53.0,100.0,80.0,Heavy thunderstorm with hail,22.94,N/A,DEL-80-71-26062505045934,2025-06-26 05:04:59.348370
1,Mumbai,28.27,33.02,26.09,88.0,18280.0,18.63,98.0,0.8,0.0,0.8,Y,0.0,0.0,N,0.8,Y,70.0,100.0,999.3,998.2,76.0,100.0,80.0,Heavy thunderstorm with hail,93.83,N/A,MUM-80-58-26062505045934,2025-06-26 05:04:59.348370
2,Bengaluru,21.46,23.76,20.46,94.0,24140.0,12.31,3.0,0.0,0.0,0.0,N,0.0,0.0,N,0.0,Y,79.0,100.0,1006.4,907.6,52.0,100.0,3.0,Overcast,5200.00,N/A,BLR-03-43-26062505045934,2025-06-26 05:04:59.348370
3,Hyderabad,24.41,27.55,22.11,87.0,24140.0,12.68,5.0,0.0,0.0,0.0,N,0.0,0.0,N,0.0,Y,61.0,100.0,1001.7,944.2,65.0,100.0,3.0,Overcast,6500.00,N/A,HYD-03-13-26062505045934,2025-06-26 05:04:59.348370
4,Chennai,27.78,33.01,23.80,79.0,24140.0,5.09,0.0,0.0,0.0,0.0,N,0.0,0.0,N,0.0,Y,0.0,100.0,1002.6,1001.3,39.0,100.0,3.0,Overcast,3900.00,N/A,CHE-03-09-26062505045934,2025-06-26 05:04:59.348370
5,Kolkata,27.11,32.96,25.69,92.0,24140.0,9.23,55.0,0.4,0.0,0.4,Y,0.0,0.0,N,0.4,Y,65.0,100.0,999.9,998.8,31.0,100.0,80.0,Heavy thunderstorm with hail,75.61,N/A,KOL-80-30-26062505045934,2025-06-26 05:04:59.348370
6,Ahmedabad,26.71,33.48,25.66,94.0,20680.0,2.74,73.0,0.9,0.0,0.9,Y,0.0,0.0,N,0.9,Y,56.0,100.0,997.6,991.6,59.0,100.0,80.0,Heavy thunderstorm with hail,64.84,N/A,AMD-80-71-26062505045934,2025-06-26 05:04:59.348370
7,Pune,24.07,27.87,21.58,86.0,24140.0,6.13,0.0,0.0,0.0,0.0,N,0.0,0.0,N,0.0,Y,80.0,100.0,1001.2,938.2,58.0,100.0,3.0,Overcast,5800.00,N/A,PUN-03-15-26062505045934,2025-06-26 05:04:59.348370
8,Jaipur,26.74,32.12,25.33,92.0,24140.0,10.82,10.0,0.0,0.0,0.0,N,0.0,0.0,N,0.0,Y,79.0,100.0,998.9,951.3,23.0,100.0,3.0,Overcast,2300.00,N/A,JAI-03-25-26062505045934,2025-06-26 05:04:59.348370
9,Lucknow,28.37,34.56,25.99,87.0,24140.0,8.29,60.0,0.1,0.0,0.1,Y,0.0,0.0,N,0.1,Y,55.0,66.0,999.5,986.4,20.0,9.0,2.0,Partly cloudy,181.82,N/A,LKO-02-10-26062505045934,2025-06-26 05:04:59.348370


In [122]:
df['unique_id']

0    DEL-80-30-26062504531457
1    MUM-80-40-26062504531457
2    BLR-03-17-26062504531457
3    HYD-03-68-26062504531457
4    CHE-03-55-26062504531457
5    KOL-80-65-26062504531457
6    AMD-80-61-26062504531457
7    PUN-03-18-26062504531457
8    JAI-03-41-26062504531457
9    LKO-02-97-26062504531457
Name: unique_id, dtype: object